In [1]:
import pandas as pd
import numpy as np
import re
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt

# **********************************************************************

In [2]:
tables=camelot.read_pdf("axis03.pdf",flavor='stream' ,pages = "all",edge_tol=500)            
if len(tables) !=0 :
    df=pd.DataFrame();    tmp=pd.DataFrame(); 
    for i in range(len(tables)) :
        tmp=tables[i].df
        if (tmp.shape[1] == 8):   
            df=pd.concat([df,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
        elif (tmp.shape[1] == 6):
            tmp=tables[2].df
            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
            tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
            tmp.columns = ["A","B","C","D","E","F"]
            tmp.reset_index(drop = True, inplace=True)
            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
            result = pd.concat([tmp1, tmp], axis=1, sort=False)
            result.drop('A' ,axis=1, inplace=True)
            result["xx"] = result["Z1"]
            for i,j in enumerate(result["xx"]):
                if re.search(r'(\d+-\d+-\d+)',j):
                    result.at[i, 'xx'] = ""
                    pass
                elif re.search(r'^\s*$',j):
                    result.at[i, 'xx'] = ""
                    pass
                else:
                    pass
                    
            for i,j in enumerate(result["Z1"]):
                if re.search(r'(\d+-\d+-\d+)',j):
                    pass
                elif re.search(r'^\s*$',j):
                    result.at[i, 'Z1'] = ""
                    pass
                else:
                    result.at[i, 'Z1'] = ""
                    pass 
            result.fillna(value=pd.np.nan, inplace=True)
            result['X'] = result['Z3'].fillna('') + result['xx'].fillna('')
            result.drop(['Z3','xx'] ,axis=1, inplace=True)
            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
            result.columns = [0,1,2,3,4,5,6,7]
            df=pd.concat([df,result]).drop_duplicates(keep='first').reset_index(drop=True)
            
    print("Completed 1")
print("Completed2")

Completed 1
Completed2


In [3]:
df.to_csv("final_axis03.csv", index=0)

# **********************************************************************

In [2]:
tables=camelot.read_pdf("axis03.pdf",flavor='stream' ,pages = "all",edge_tol=500)            
if len(tables) !=0 :
    df=pd.DataFrame();    tmp=pd.DataFrame(); 
    for i in range(len(tables)) :
        tmp=tables[i].df  
        df=pd.concat([df,tmp]).drop_duplicates(keep='first').reset_index(drop=True)

In [3]:
tmp=tables[2].df

In [4]:
idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
print("Columns:",tmp.columns)

tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
tmp=pd.DataFrame(tmp.T.drop_duplicates().T)

Columns: Index(['Tran Date\nValue Date\nTransaction Particulars', 'Chq No.', 'Amount',
       'DR|CR', 'Balance', 'Branch Name'],
      dtype='object', name=0)


In [5]:
tmp.columns = ["A","B","C","D","E","F"]

In [6]:
tmp.reset_index(drop = True, inplace=True)

In [7]:
tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])

In [8]:
result = pd.concat([tmp1, tmp], axis=1, sort=False)

In [9]:
result.drop('A' ,axis=1, inplace=True)

In [10]:
result["xx"] = result["Z1"]

In [11]:
result.to_csv("result.csv", index=0)

In [12]:
for i,j in enumerate(result["xx"]):
    if re.search(r'(\d+-\d+-\d+)',j):
        result.at[i, 'xx'] = ""
        pass
    elif re.search(r'^\s*$',j):
        result.at[i, 'xx'] = ""
        pass
    else:
        print(j)
        
for i,j in enumerate(result["Z1"]):
    if re.search(r'(\d+-\d+-\d+)',j):
        pass
    elif re.search(r'^\s*$',j):
        result.at[i, 'Z1'] = ""
        pass
    else:
        result.at[i, 'Z1'] = ""
        pass 

BY CASH DEPOSIT-
BNA/DPRH329801/6971/170119/SOUTH D
BY CASH DEPOSIT-
BNA/DPRH329801/8113/230119/SOUTH D
BY CASH DEPOSIT-
BNA/DPRH329801/8115/230119/SOUTH D
BY CASH DEPOSIT-
BNA/DPRH329801/8913/280119/SOUTH D


In [13]:
# result

In [14]:
result.fillna(value=pd.np.nan, inplace=True)

In [15]:
result['X'] = result['Z3'].fillna('') + result['xx'].fillna('')

In [16]:
# result.to_csv("result.csv",index=0)

In [17]:
result.drop(['Z3','xx'] ,axis=1, inplace=True)

In [18]:
result.to_csv("result.csv",index=0)

In [24]:
result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
result.to_csv("result.csv",index=0)

In [25]:
result.columns = ["Tran Date","Value Date","Transaction Particulars","Chq No","Amount","DR|CR","Balance","Branch Name"]

In [26]:
result.to_csv("result.csv",index=0)